### STEP : Diversity Analysis



#### Example

- [“Moving Pictures” tutorial](https://docs.qiime2.org/2022.8/tutorials/moving-pictures-usage/?highlight=ancom)
- [Biostats ANCOM](http://scikit-bio.org/docs/0.4.2/generated/generated/skbio.stats.composition.ancom.html)


#### Methods
- [composition](https://docs.qiime2.org/2022.8/plugins/available/composition/)
- [composition add_pseudocount](https://docs.qiime2.org/2022.8/plugins/available/composition/add-pseudocount/): Increment all counts in table by pseudocount.
- [composition ancom](https://docs.qiime2.org/2022.8/plugins/available/composition/ancom/): Apply ANCOM to identify features that differ in abundance.

## Setup and settings

In [1]:
# Importing packages
import os
import pandas as pd
from qiime2 import Artifact
from qiime2 import Visualization
from qiime2 import Metadata

from qiime2.plugins.composition.visualizers import ancom
from qiime2.plugins.composition.methods import add_pseudocount

from qiime2.plugins.feature_table.methods import filter_samples
from qiime2.plugins.feature_table.methods import filter_seqs

%matplotlib inline

### Receiving the parameters

The following cell can receive parameters using the [papermill](https://papermill.readthedocs.io/en/latest/) tool.

In [2]:
metadata_file = '/home/lauro/nupeb/rede-micro/redemicro-miliane-nutri/data/raw/metadata/miliane-metadata-CxAC.tsv'
base_dir = os.path.join('/', 'home', 'lauro', 'nupeb', 'rede-micro', 'redemicro-miliane-nutri')
experiment_name = 'miliane-CxAC-trim'
class_col = 'group-id'
replace_files = False

In [3]:
# Parameters
PAPERMILL_INPUT_PATH = "nb-templates/step-abundance-analysis2.ipynb"
PAPERMILL_OUTPUT_PATH = "/home/lauro/nupeb/rede-micro/redemicro-ana-flavia-nutri/experiments/ana-flavia-NCxNR-trim/nb-executed-steps/step-abundance-analysis-ana-flavia-NCxNR-trim.ipynb"
experiment_name = "ana-flavia-NCxNR-trim"
base_dir = "/home/lauro/nupeb/rede-micro/redemicro-ana-flavia-nutri"
manifest_file = "/home/lauro/nupeb/rede-micro/redemicro-ana-flavia-nutri/data/raw/manifest/manifest-ana-flavia-NCxNR.csv"
metadata_file = "/home/lauro/nupeb/rede-micro/redemicro-ana-flavia-nutri/data/raw/metadata/metadata-ana-flavia-NCxNR.tsv"
class_col = "group-id"
classifier_file = "/home/lauro/nupeb/rede-micro/models/silva-138-99-nb-classifier.qza"
top_n = 20
replace_files = False
phred = 20
trunc_f = 0
trunc_r = 0
overlap = 12
threads = 6
trim = {
    "overlap": 8,
    "forward_primer": "CCTACGGGRSGCAGCAG",
    "reverse_primer": "GGACTACHVGGGTWTCTAAT",
}


In [4]:
experiment_folder = os.path.abspath(os.path.join(base_dir, 'experiments', experiment_name))
img_folder = os.path.abspath(os.path.join(experiment_folder, 'imgs'))

### Defining names, paths and flags

In [5]:
# QIIME2 Artifacts folder
qiime_folder = os.path.join(experiment_folder, 'qiime-artifacts')

# Input - DADA2 Artifacts
dada2_tabs_path = os.path.join(qiime_folder, 'dada2-tabs.qza')

## Step execution

### Load input files

This Step import the QIIME2 `FeatureTable[Frequency]` Artifact and the `Metadata` file.

In [6]:
#Load Metadata
metadata_qa = Metadata.load(metadata_file)

#Load FeatureTable[Frequency]
tabs = Artifact.load(dada2_tabs_path)

# Filter FeatureTable[Frequency | RelativeFrequency | PresenceAbsence | Composition] based on Metadata sample ID values
tabs = filter_samples(
    table=tabs,
    metadata=metadata_qa,
).filtered_table

{'min_frequency': 0, 'max_frequency': None, 'min_features': 0, 'max_features': None, 'metadata': Metadata
--------
9 IDs x 3 columns
sample-name: ColumnProperties(type='categorical')
group-id:    ColumnProperties(type='categorical')
group-desc:  ColumnProperties(type='categorical')

Call to_dataframe() for a tabular representation., 'where': None, 'exclude_ids': False, 'filter_empty_features': True, 'table': 2732 x 74 <class 'biom.table.Table'> with 12512 nonzero entries (6% dense)}
{'min_frequency': 0, 'max_frequency': None, 'min_features': 0, 'max_features': None, 'metadata': Metadata
--------
9 IDs x 3 columns
sample-name: ColumnProperties(type='categorical')
group-id:    ColumnProperties(type='categorical')
group-desc:  ColumnProperties(type='categorical')

Call to_dataframe() for a tabular representation., 'where': None, 'exclude_ids': False, 'filter_empty_features': True, 'table': 2732 x 74 <class 'biom.table.Table'> with 12512 nonzero entries (6% dense)}


# ANCOM

Apply Analysis of Composition of Microbiomes (ANCOM) to identify features
that are differentially abundant across groups.

- [composition add_pseudocount](https://docs.qiime2.org/2022.8/plugins/available/composition/add-pseudocount/): Increment all counts in table by pseudocount.
- [composition ancom](https://docs.qiime2.org/2022.8/plugins/available/composition/ancom/): Apply ANCOM to identify features that differ in abundance.

In [7]:
# Select class column
column = metadata_qa.get_column(class_col)

In [8]:
# Create the pseudocount table
composition_tab = add_pseudocount(table = tabs).composition_table

{'pseudocount': 1, 'table': 668 x 9 <class 'biom.table.Table'> with 1453 nonzero entries (24% dense)}
{'pseudocount': 1, 'table': 668 x 9 <class 'biom.table.Table'> with 1453 nonzero entries (24% dense)}


In [9]:
# Create visualizations for each parameter combination for the ANCOM  method.
transform_functions = ('sqrt', 'log', 'clr')
difference_functions = ('mean_difference', 'f_statistic')
for t in transform_functions:
    for d in difference_functions:
        print(f"Calculating ANCOM with: {t} {d}")
        try:
            ancom_viz = ancom(
                table = composition_tab, 
                metadata = column, 
                transform_function = t, 
                difference_function = d
            ).visualization
            view_name = os.path.join(qiime_folder, f'ancom-{t}-{d}.qzv')
            ancom_viz.save(view_name)
        except Exception as e:
            print(f"ERROR: Calculating ANCOM with: {t} {d}")
            print(e)

Calculating ANCOM with: sqrt mean_difference
{'metadata': <CategoricalMetadataColumn name='group-id' id_count=9>, 'transform_function': 'sqrt', 'difference_function': 'mean_difference', 'table':                c688e3b1ada46ed57f5b7e7e0d56664f  \
S210421121673                            7750.0   
S210421121674                             205.0   
S210421121675                           32289.0   
S210421121676                           87774.0   
S210421121677                           30176.0   
S210421121678                            3067.0   
S210421121679                            2083.0   
S210421121680                            1104.0   
S210421121681                            4242.0   

               bc798de4a9acd3ff7ba51c244523be14  \
S210421121673                           10503.0   
S210421121674                           37285.0   
S210421121675                            3541.0   
S210421121676                           68731.0   
S210421121677                          

/home/lauro/anaconda3/envs/qiime2-2022.2/lib/python3.8/site-packages/scipy/stats/_stats_py.py:3659: F_onewayConstantInputWarning: Each of the input arrays is constant;the F statistic is not defined or infinite
  warnings.warn(F_onewayConstantInputWarning())


Calculating ANCOM with: sqrt f_statistic
{'metadata': <CategoricalMetadataColumn name='group-id' id_count=9>, 'transform_function': 'sqrt', 'difference_function': 'f_statistic', 'table':                c688e3b1ada46ed57f5b7e7e0d56664f  \
S210421121673                            7750.0   
S210421121674                             205.0   
S210421121675                           32289.0   
S210421121676                           87774.0   
S210421121677                           30176.0   
S210421121678                            3067.0   
S210421121679                            2083.0   
S210421121680                            1104.0   
S210421121681                            4242.0   

               bc798de4a9acd3ff7ba51c244523be14  \
S210421121673                           10503.0   
S210421121674                           37285.0   
S210421121675                            3541.0   
S210421121676                           68731.0   
S210421121677                            3827.0

/home/lauro/anaconda3/envs/qiime2-2022.2/lib/python3.8/site-packages/scipy/stats/_stats_py.py:3659: F_onewayConstantInputWarning: Each of the input arrays is constant;the F statistic is not defined or infinite
  warnings.warn(F_onewayConstantInputWarning())


Calculating ANCOM with: log mean_difference
{'metadata': <CategoricalMetadataColumn name='group-id' id_count=9>, 'transform_function': 'log', 'difference_function': 'mean_difference', 'table':                c688e3b1ada46ed57f5b7e7e0d56664f  \
S210421121673                            7750.0   
S210421121674                             205.0   
S210421121675                           32289.0   
S210421121676                           87774.0   
S210421121677                           30176.0   
S210421121678                            3067.0   
S210421121679                            2083.0   
S210421121680                            1104.0   
S210421121681                            4242.0   

               bc798de4a9acd3ff7ba51c244523be14  \
S210421121673                           10503.0   
S210421121674                           37285.0   
S210421121675                            3541.0   
S210421121676                           68731.0   
S210421121677                            

...

/home/lauro/anaconda3/envs/qiime2-2022.2/lib/python3.8/site-packages/scipy/stats/_stats_py.py:3659: F_onewayConstantInputWarning: Each of the input arrays is constant;the F statistic is not defined or infinite
  warnings.warn(F_onewayConstantInputWarning())


Calculating ANCOM with: clr mean_difference
{'metadata': <CategoricalMetadataColumn name='group-id' id_count=9>, 'transform_function': 'clr', 'difference_function': 'mean_difference', 'table':                c688e3b1ada46ed57f5b7e7e0d56664f  \
S210421121673                            7750.0   
S210421121674                             205.0   
S210421121675                           32289.0   
S210421121676                           87774.0   
S210421121677                           30176.0   
S210421121678                            3067.0   
S210421121679                            2083.0   
S210421121680                            1104.0   
S210421121681                            4242.0   

               bc798de4a9acd3ff7ba51c244523be14  \
S210421121673                           10503.0   
S210421121674                           37285.0   
S210421121675                            3541.0   
S210421121676                           68731.0   
S210421121677                            

/home/lauro/anaconda3/envs/qiime2-2022.2/lib/python3.8/site-packages/scipy/stats/_stats_py.py:3659: F_onewayConstantInputWarning: Each of the input arrays is constant;the F statistic is not defined or infinite
  warnings.warn(F_onewayConstantInputWarning())


Calculating ANCOM with: clr f_statistic
{'metadata': <CategoricalMetadataColumn name='group-id' id_count=9>, 'transform_function': 'clr', 'difference_function': 'f_statistic', 'table':                c688e3b1ada46ed57f5b7e7e0d56664f  \
S210421121673                            7750.0   
S210421121674                             205.0   
S210421121675                           32289.0   
S210421121676                           87774.0   
S210421121677                           30176.0   
S210421121678                            3067.0   
S210421121679                            2083.0   
S210421121680                            1104.0   
S210421121681                            4242.0   

               bc798de4a9acd3ff7ba51c244523be14  \
S210421121673                           10503.0   
S210421121674                           37285.0   
S210421121675                            3541.0   
S210421121676                           68731.0   
S210421121677                            3827.0  

/home/lauro/anaconda3/envs/qiime2-2022.2/lib/python3.8/site-packages/scipy/stats/_stats_py.py:3659: F_onewayConstantInputWarning: Each of the input arrays is constant;the F statistic is not defined or infinite
  warnings.warn(F_onewayConstantInputWarning())
